In [10]:
#使用协程解决（单线程+多任务异步协程）
#协程： 可以把协程当作一个特别的函数，如果一个函数的定义被async所修饰,就不会立即执行，而是返回协程对象
#由于requests当中的代码不可以出现在特殊函数内部，requests不支持异步操作。所以要引入aiohttp
import requests
from lxml import etree
import nest_asyncio  #针对jupyternotebook错误情况~
import asyncio
import time
import aiohttp
nest_asyncio.apply()



#待执行函数中不可以使用不支持异步模块的代码.在每一个with前加async  在每一个阻塞操作上加await
async def get_requests1(url,headers):
    #必须用with as结构声明
    async with aiohttp.ClientSession() as s:
        async with await s.get(url=url,headers=headers) as response:
            response.encoding = 'utf-8'
            #response.read()  是bytes
            page_text = await response.text()
            return [page_text,url[-2]]

#回调函数
def callback1(task):
    result = task.result()[0]
    page_num = task.result()[1]
    tree = etree.HTML(result)
    data = tree.xpath('//div[@class="content"]/span[1]')
    xpath_out = []
    for i in data:
        text1 = i.xpath("string(.)").strip()
        xpath_out.append(text1)
    

if __name__ == '__main__':
    
    url = 'https://www.qiushibaike.com/text/page/%d/'
    urls = []
    num = int(input('请输入爬取页数：'))
    for i in range(1,num+1):
        urls.append(format(url%i))
    
    tasks = [] #用于放封装了的任务对象

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'
    }

    for i in urls:
        c = get_requests1(i,headers) #接收协程对象

        #任务对象(task)： 对协程对象作进一步的封装！ 比如显示协程运行情况
        task = asyncio.ensure_future(c)
        task.add_done_callback(callback1) #绑定回调函数
        tasks.append(task)

    #注册到事件循环对象中,事件循环对象中可以放很多任务对象去执行。 异步体现在开始时是顺序执行任务对象，而一个任务事件发生阻塞不会等待，而是继续执行
    #下一个对象
    time1 = time.time()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(asyncio.wait(tasks)) #每一个任务对象，进行挂起操作。在阻塞时就自动挂起
    time2 = time.time()
    print(time2-time1)




请输入爬取页数：13
0.31709718704223633
